In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [2]:
# Load datasets
combined_news = pd.read_csv('/content/drive/MyDrive/GoogleGirlsScript/286Dailynews/Combined_News_DJIA.csv')
reddit_news = pd.read_csv('/content/drive/MyDrive/GoogleGirlsScript/286Dailynews/RedditNews.csv')
djia_data = pd.read_csv('/content/drive/MyDrive/GoogleGirlsScript/286Dailynews/upload_DJIA_table.csv')


In [3]:
# Combine top news headlines into a single column
combined_news['Combined_News'] = combined_news.iloc[:, 2:].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
combined_news = combined_news[['Date', 'Label', 'Combined_News']]

In [4]:
# Merge with DJIA data
djia_data['Date'] = pd.to_datetime(djia_data['Date'])
combined_news['Date'] = pd.to_datetime(combined_news['Date'])

<ipython-input-4-005cb4616053>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_news['Date'] = pd.to_datetime(combined_news['Date'])


In [5]:
data = pd.merge(combined_news, djia_data, on='Date')

In [6]:
# Feature extraction from text data
vectorizer = TfidfVectorizer(max_features=5000)
news_tfidf = vectorizer.fit_transform(data['Combined_News']).toarray()

In [7]:
# Combine with stock data
stock_features = data[['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']].values
scaler = StandardScaler()
stock_features_scaled = scaler.fit_transform(stock_features)

In [8]:
X = np.hstack((news_tfidf, stock_features_scaled))
y = data['Label'].values

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2



In [11]:
# Define the model with regularization and dropout
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [15]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
25/25 [==============================] - 3s 85ms/step - loss: 6.4215 - accuracy: 0.5110 - val_loss: 2.4116 - val_accuracy: 0.5729
Epoch 2/100
25/25 [==============================] - 2s 81ms/step - loss: 1.6591 - accuracy: 0.5236 - val_loss: 1.1852 - val_accuracy: 0.5678
Epoch 3/100
25/25 [==============================] - 2s 79ms/step - loss: 0.9891 - accuracy: 0.5223 - val_loss: 0.8330 - val_accuracy: 0.5578
Epoch 4/100
25/25 [==============================] - 1s 56ms/step - loss: 0.7762 - accuracy: 0.5261 - val_loss: 0.7279 - val_accuracy: 0.5704
Epoch 5/100
25/25 [==============================] - 1s 46ms/step - loss: 0.7152 - accuracy: 0.5374 - val_loss: 0.7002 - val_accuracy: 0.5553
Epoch 6/100
25/25 [==============================] - 1s 49ms/step - loss: 0.6994 - accuracy: 0.5286 - val_loss: 0.6927 - val_accuracy: 0.5603
Epoch 7/100
25/25 [==============================] - 1s 58ms/step - loss: 0.6954 - accuracy: 0.5349 - val_loss: 0.6904 - val_accuracy: 0.5653
Epoch 

In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

13/13 [==============================] - 0s 7ms/step - loss: 0.6883 - accuracy: 0.5729
Test Accuracy: 57.29%


In [17]:
# Predict and calculate additional metrics if necessary
from sklearn.metrics import classification_report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

13/13 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

           0       1.00      0.01      0.01       171
           1       0.57      1.00      0.73       227

    accuracy                           0.57       398
   macro avg       0.79      0.50      0.37       398
weighted avg       0.76      0.57      0.42       398



In [19]:
model.save('stock_prediction_model.h5')
